# Introduction

Collect relevant structured data for all labeled patients, for the purpose of creating classifiers separate from the NLP project, to identify patient cohorts to preferrentially sample for manual concept analysis.

**Authors**
- Eric Carlson

In [1]:
from datetime import datetime
import configparser
import hashlib
from importlib import reload
import logging
import numpy as np
import os
import pandas as pd
import pathlib as pl
import sys
import yaml

from IPython import display

import etc_utils as eu
import mimic_extraction_utils as meu
import structured_data_utils as sdu

In [2]:
import sqlalchemy as sa
from sqlalchemy.engine import reflection
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base, name_for_scalar_relationship, generate_relationship

In [3]:
reload(eu)
reload(meu)
reload(sdu)

<module 'structured_data_utils' from '/mnt/cbds_homes/ecarlson/Notebooks/mit_frequent_fliers/mit-team-code/software/notebooks/structured_data_utils.py'>

In [4]:
sys.path.append('../icd9')
from icd9 import ICD9

# feel free to replace with your path to the json file
tree = ICD9('../icd9/codes.json')

## Configure pandas and matplot lib for nice web printing

In [5]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 100

In [6]:
%matplotlib inline

## Load config files, configure logging

In [7]:
work_desc = "gather_structured_data"

In [8]:
time_str, path_config, creds = eu.load_config_v2()
print('Time string: {}'.format(time_str))

print('Paths:')
for k, item in path_config.items():
    print('{}: {}'.format(k, item))

Time string: 2016-10-24-16-35
Paths:
input_dir: /mnt/cbds_homes/ecarlson/data/mit_frequent_fliers
repo_data_dir: ../../data
results_dir: /mnt/cbds_homes/ecarlson/results/mit_frequent_fliers
log_dir: /mnt/cbds_homes/ecarlson/logs/mit_frequent_fliers


In [9]:
logger = logging.getLogger()

eu.configure_logging(logger, work_desc=work_desc, log_directory=path_config['log_dir'], time_str=time_str)

Logging to /mnt/cbds_homes/ecarlson/logs/mit_frequent_fliers/2016-10-24-16-35_gather_structured_data.log


In [10]:
[k for k in creds.keys()]

['mimic3_v1_4', 'mimic2_v2_6', 'postgres']

# Connect to database

In [11]:
db_path = '{sa_driver}://{username}:{password}@{hostname}/{dbname}'.format(**creds['mimic3_v1_4'])
engine = create_engine(db_path)
sm = sessionmaker(bind=engine)
s = sm()
conn = s.connection()

meta = MetaData(schema="mimiciii", bind=engine)
meta.reflect(bind=engine)

In [12]:
base = automap_base(metadata=meta)
base.prepare(engine, reflect=True)

In [13]:
for cls_name in sorted(base.classes.keys()):
    print(cls_name)

admissions
callout
caregivers
chartevents
cptevents
d_cpt
d_icd_diagnoses
d_icd_procedures
d_items
d_labitems
datetimeevents
diagnoses_icd
drgcodes
icustays
inputevents_cv
inputevents_mv
labevents
microbiologyevents
noteevents
outputevents
patients
prescriptions
procedureevents_mv
procedures_icd
services
transfers


In [14]:
note_tb = base.classes['noteevents']

In [15]:
s.query(note_tb.category).count()

2083180

# Load labeled notes

In [16]:
categories = ['Advanced.Cancer', 'Advanced.Heart.Disease', 'Advanced.Lung.Disease',
       'Alcohol.Abuse',
       'Chronic.Neurological.Dystrophies', 'Chronic.Pain.Fibromyalgia',
       'Dementia', 'Depression', 'Developmental.Delay.Retardation',
       'Non.Adherence', 'None',
       'Obesity', 'Other.Substance.Abuse', 
       'Schizophrenia.and.other.Psychiatric.Disorders', 'Unsure',]

In [17]:
data_path = pl.Path(path_config['repo_data_dir'])

In [18]:
[p for p in data_path.glob('*csv')]

[PosixPath('../../data/mimic3_note_metadata_2016-10-24-00-37.csv'),
 PosixPath('../../data/notes_icd9_codes_2016-10-24-00-37.csv'),
 PosixPath('../../data/combined_label_data_2016-10-24-00-37.csv'),
 PosixPath('../../data/mimic3_note_equivs_2016-10-22-03-39.csv'),
 PosixPath('../../data/nursingNotesClean.csv'),
 PosixPath('../../data/AllDischargeFinal27Sep16.csv'),
 PosixPath('../../data/dischargeSummariesClean.csv'),
 PosixPath('../../data/AllnursingFinal27Sep16.csv')]

In [19]:
nursing_notes_path = data_path.joinpath('nursingNotesClean.csv')
discharge_notes_path = data_path.joinpath('dischargeSummariesClean.csv')

In [20]:
nursing_notes = pd.read_csv(nursing_notes_path.as_posix())
disch_notes = pd.read_csv(discharge_notes_path.as_posix()).rename(columns={'subject.id':'subject_id'})

In [21]:
display.display(nursing_notes.head(1))
print(nursing_notes.loc[0,'text'])

,Hospital.Admission.ID,subject_id,icustay_id,charttime,realtime,category,title,text,cohort,rand,Subject.ID,ICU.ID,Note.Type,Chart.time,Category,Real.time,None,Obesity,Non.Adherence,Developmental.Delay.Retardation,Advanced.Heart.Disease,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure,operator
0,2,24807,30800.0,3033-07-09 00:56:00,3033-07-09 01:22:00,Nursing/Other,MICU/SICU NURSING PROGRESS NOTE.,\nMICU/SICU NURSING PROGRESS NOTE.\n SEE CAREVIEW FOR OBJECTIVE DATA.\n Neuro: Arouses w...,1,0.330879,24807,30800.0,Nursing/Other,3033-07-09 00:56:00,Nursing/Other,3033-07-09 01:22:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ETM



MICU/SICU NURSING PROGRESS NOTE.
     SEE CAREVIEW FOR OBJECTIVE DATA.
     Neuro: Arouses with verbal stimuli, ms is variable and changes from a&o x 3 to a&o x 1 with moaning episodes, repetitive speach and repetative motions with rue.  Pupils are rt 4mm, lt 3mm and sluggish bilat.  Head ct completed and was negative.  Was given versed 1 mg during ct and pt became much more calm and was able to answer questions appropriatly. Reports able to feel in all 4 extrem., Partial movement of rt ue and lt ue.
     Respiratory:  Lung sounds are coarse throughout, diminished in lt base.  RR 12-24 and non-labored except when having moaning episodes.  O2 saturation is 94-100% on 4l nc.  Cxr in ed showed persistent lll pneumonia and atelectasis in rt upper field.  Expectorating thick tan sputum in abundant amounts.
     CV:  Sinus rythm, rate 70-98 with no ectopy noted.  Nbp 104- 134 systolic.  Good pulses all 4 extrem.  
      GI/GU:  Abdomen si softly distended with + bs.  Pt is able to take regu

In [22]:
display.display(disch_notes.head(1))
#print(disch_notes.loc[0,'text'])

,Hospital.Admission.ID,subject_id,chartdate,category,descriptions,text,cohort,Subject.ID,ICU.ID,Note.Type,Chart.time,Category,Real.time,None,Obesity,Non.Adherence,Developmental.Delay.Retardation,Advanced.Heart.Disease,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure,operator
0,100020,9973,10-DEC-2142 00:00:00,Discharge,Summary,\n\n\n\nAdmission Date: [**2512-1-8**] Discharge Date: [**2512-1-18**]\n\nDate o...,1,9973,100020,Discharge,100020,Discharge,10-DEC-2142 00:00:00,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,JTW


In [23]:
mimic3_map = pd.read_csv(data_path.joinpath('mimic3_note_equivs_2016-10-22-03-39.csv').as_posix())

In [24]:
mimic3_map.head(5)

,subject_id,hadm_id,icustay_id,category,chartdate,charttime,realtime,length,md5,str_start,str_end,row_id_m3,chartdate_m3,charttime_m3,storetime_m3,md5_m3,str_start_m3,str_end_m3,len_diff_pct,str_start_diff,str_end_diff,total_m3_distance
0,68.0,17608.0,78.0,Nursing/other,NaN,2728-12-13 16:51:00,2728-12-13 17:08:00,1666.0,27572b36bd4c26c322f50cf65d095d16,NSG0700-1900Events:Attemptedeatingbreakfastbutbeca,ine..?doubleorsinglelumen.Continuetomonitoroutput.,1261174.0,2173-12-16,2173-12-16 16:51:00,2173-12-16 17:08:00,a71a1009932ab1fce83b8867234ec9e4,NSG0700-1900Events:Attemptedeatingbreakfastbutbeca,ine..?doubleorsinglelumen.Continuetomonitoroutput.,0.001200,0.00,0.0,0.001200
1,109.0,164029.0,NaN,Discharge summary,2142-12-10,NaN,2142-12-10 00:00:00,7751.0,f5f69772c32f1b0ac05b7cf408f7a6db,:::Sex:FService:MEDICINEAllergies:PenicillinsAtten,"rativeassistant,,atifyouneedtoconfirmorreschedule.",15322.0,2140-01-21,NaN,NaN,1770e41dbbe1a89a144bcebfad1fc87e,:::Sex:FService:MEDICINEAllergies:PenicillinsAtten,"rativeassistant,,atifyouneedtoconfirmorreschedule.",0.002580,0.00,0.0,0.002580
2,109.0,26693.0,132.0,Nursing/other,NaN,3353-03-09 02:13:00,3353-03-09 02:47:00,2107.0,27d1f5907fa14b6702837a845f84c54e,ficunsgnote:21:30-7:00thisisa22y.o.womanadmwithlup,needtocontinuehemodialysisscheduleqtues/thurs/sat.,1261605.0,2140-04-08,2140-04-08 03:13:00,2140-04-08 03:47:00,6e88c96f2f3f7dab74d3732eec27dd35,nsgnote:21:30-7:00thisisa22y.o.womanadmwithlupusdx,needtocontinuehemodialysisscheduleqtues/thurs/sat.,0.031799,0.08,0.0,0.111799
3,109.0,12757.0,130.0,Nursing/other,NaN,3352-12-19 16:48:00,3352-12-19 17:05:00,1294.0,3e0fff775cfb678fdfa06ece68ebfab5,NPN7A-7PSeecarevueforspecifics:Briefly22yroldfemal,"wnwhendialysiscompleted,LabetalolgttforSBP>180prn.",1261597.0,2140-01-19,2140-01-19 16:48:00,2140-01-19 17:05:00,d3350e7e8b6a9073feae2c0f11ea82e3,NPN7A-7PSeecarevueforspecifics:Briefly22yroldfemal,"wnwhendialysiscompleted,LabetalolgttforSBP>180prn.",0.000773,0.00,0.0,0.000773
4,109.0,14234.0,129.0,Nursing/other,NaN,3351-03-18 02:36:00,3351-03-18 03:24:00,3083.0,8efc0a2ff698b75ce183e3183c1bf204,MICUA7P-7ASHIFTSUMMARYOFEVENTS:PTADMITTEDAT1930INH,byremovingclonodinepatches.tore-evalanti-hypertens,1261589.0,2138-04-17,2138-04-17 03:36:00,2138-04-17 04:24:00,b26423d357dc3009f6e063c405ce742e,MICUA7P-7ASHIFTSUMMARYOFEVENTS:PTADMITTEDATINHYPER,byremovingclonodinepatches.tore-evalanti-hypertens,0.013947,0.08,0.0,0.093947


In [25]:
disch_notes['md5'] = disch_notes['text'].apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())
nursing_notes['md5'] = nursing_notes['text'].apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())

In [26]:
cols_to_keep = ['subject_id', 'category', 'md5', 'operator'] + categories
comb_dat = pd.concat([disch_notes[cols_to_keep], nursing_notes[cols_to_keep]])

In [27]:
comb_dat.head()

,subject_id,category,md5,operator,Advanced.Cancer,Advanced.Heart.Disease,Advanced.Lung.Disease,Alcohol.Abuse,Chronic.Neurological.Dystrophies,Chronic.Pain.Fibromyalgia,Dementia,Depression,Developmental.Delay.Retardation,Non.Adherence,None,Obesity,Other.Substance.Abuse,Schizophrenia.and.other.Psychiatric.Disorders,Unsure
0,9973,Discharge,56f2598342cce321539d8975809d487c,JTW,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
1,9973,Discharge,56f2598342cce321539d8975809d487c,ETM,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
2,3365,Discharge,eaea5c3c7577135a83f1f0fb583e0d53,JTW,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,3365,Discharge,eaea5c3c7577135a83f1f0fb583e0d53,ETM,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,27290,Discharge,e7433c0b75ea00346390f029bb830774,JW,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [28]:
comb_dat.tail()

,subject_id,category,md5,operator,Advanced.Cancer,Advanced.Heart.Disease,Advanced.Lung.Disease,Alcohol.Abuse,Chronic.Neurological.Dystrophies,Chronic.Pain.Fibromyalgia,Dementia,Depression,Developmental.Delay.Retardation,Non.Adherence,None,Obesity,Other.Substance.Abuse,Schizophrenia.and.other.Psychiatric.Disorders,Unsure
1994,28068,Nursing/Other,de1d6912eb489b0ab1cb35137c69e995,ETM,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1995,28068,Nursing/Other,de1d6912eb489b0ab1cb35137c69e995,JTW,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1996,31369,Nursing/Other,2302d0a9ce57a278e84dbbb04f599089,JF,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1997,31369,Nursing/Other,2302d0a9ce57a278e84dbbb04f599089,JW,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1998,31369,Nursing/Other,2302d0a9ce57a278e84dbbb04f599089,JF,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [29]:
comb_dat = pd.merge(comb_dat, mimic3_map[['md5', 'row_id_m3', 'total_m3_distance']], on='md5', how='left')

In [30]:
comb_dat.head()

,subject_id,category,md5,operator,Advanced.Cancer,Advanced.Heart.Disease,Advanced.Lung.Disease,Alcohol.Abuse,Chronic.Neurological.Dystrophies,Chronic.Pain.Fibromyalgia,Dementia,Depression,Developmental.Delay.Retardation,Non.Adherence,None,Obesity,Other.Substance.Abuse,Schizophrenia.and.other.Psychiatric.Disorders,Unsure,row_id_m3,total_m3_distance
0,9973,Discharge,56f2598342cce321539d8975809d487c,JTW,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,NaN,NaN
1,9973,Discharge,56f2598342cce321539d8975809d487c,ETM,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,NaN,NaN
2,3365,Discharge,eaea5c3c7577135a83f1f0fb583e0d53,JTW,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,17170.0,0.000884
3,3365,Discharge,eaea5c3c7577135a83f1f0fb583e0d53,ETM,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,17170.0,0.000884
4,27290,Discharge,e7433c0b75ea00346390f029bb830774,JW,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,50828.0,0.001554


In [31]:
grouped = comb_dat.groupby('md5')['total_m3_distance'].last()
pct_unmatched = grouped.isnull().sum()/grouped.count() * 100
print('{:3.2}% of notes had no exact match in MIMIC 3'.format(pct_unmatched))

2.6% of notes had no exact match in MIMIC 3


In [56]:
output_path = pl.Path(path_config['repo_data_dir']).joinpath('combined_label_data_{}.csv'.format(time_str))
logger.info(output_path)
comb_dat.to_csv(output_path.as_posix(), index=False)

2016-10-24 16:41:49,493 - root - INFO - ../../data/combined_label_data_2016-10-24-16-35.csv


# Gather patient's data from the database and export

## Test functions for gathering data

In [33]:
note_meta = sdu.get_note_metadata(conn, comb_dat.loc[2, 'row_id_m3'])

In [34]:
for (k, v) in note_meta.items():
    print("{}:\t{}".format(k, v))

cgid:	None
charttime:	None
storetime:	None
chartdate:	2179-12-23 00:00:00
subject_id:	3365
hadm_id:	100103


In [35]:
note_meta.keys()

dict_keys(['cgid', 'charttime', 'storetime', 'chartdate', 'subject_id', 'hadm_id'])

In [36]:
diags = sdu.get_hadm_diagnoses(conn, note_meta['hadm_id'])

In [37]:
diags[:3]

[{'clean_icd9_code': '396.3',
  'hadm_id': 100103,
  'icd9_code': '3963',
  'known_icd9_code': True,
  'long_title': 'Mitral valve insufficiency and aortic valve insufficiency',
  'seq_num': 1,
  'short_title': 'Mitral/aortic val insuff',
  'subject_id': 3365},
 {'clean_icd9_code': '398.91',
  'hadm_id': 100103,
  'icd9_code': '39891',
  'known_icd9_code': True,
  'long_title': 'Rheumatic heart failure (congestive)',
  'seq_num': 2,
  'short_title': 'Rheumatic heart failure',
  'subject_id': 3365},
 {'clean_icd9_code': '427.31',
  'hadm_id': 100103,
  'icd9_code': '42731',
  'known_icd9_code': True,
  'long_title': 'Atrial fibrillation',
  'seq_num': 3,
  'short_title': 'Atrial fibrillation',
  'subject_id': 3365}]

In [38]:
[d['clean_icd9_code'] for d in diags if d['known_icd9_code']]

['396.3',
 '398.91',
 '427.31',
 '414.01',
 '401.9',
 '272.4',
 '530.81',
 'V10.11',
 'V15.2']

In [39]:
sdu.print_icd9_tree(diags[0]['clean_icd9_code'])

Parents:
- ROOT: ROOT
- 390-459: DISEASES OF THE CIRCULATORY SYSTEM 
- 393-398: CHRONIC RHEUMATIC HEART DISEASE 
- 396: Diseases of mitral and aortic valves
- 396.3: Mitral valve insufficiency and aortic valve insufficiency

-> 396.3: Mitral valve insufficiency and aortic valve insufficiency

Children:


In [40]:
sdu.print_icd9_tree(diags[5]['clean_icd9_code'])

Parents:
- ROOT: ROOT
- 240-279: ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS 
- 270-279: OTHER METABOLIC AND IMMUNITY DISORDERS 
- 272: Disorders of lipoid metabolism
- 272.4: Other and unspecified hyperlipidemia

-> 272.4: Other and unspecified hyperlipidemia

Children:


In [41]:
sdu.print_icd9_tree(diags[7]['clean_icd9_code'])

Parents:
- ROOT: ROOT
- V10-V19: PERSONS WITH POTENTIAL HEALTH HAZARDS RELATED TO PERSONAL AND FAMILY HISTORY 
- V10: Personal history of malignant neoplasm
- V10.1: Trachea, bronchus, and lung
- V10.11: Bronchus and lung

-> V10.11: Bronchus and lung

Children:


In [42]:
sdu.print_icd9_tree(diags[8]['clean_icd9_code'])

Parents:
- ROOT: ROOT
- V10-V19: PERSONS WITH POTENTIAL HEALTH HAZARDS RELATED TO PERSONAL AND FAMILY HISTORY 
- V15: Other personal history presenting hazards to health
- V15.2: Surgery to other organs

-> V15.2: Surgery to other organs

Children:
- V15.29: Surgery to other organs
- V15.22: Personal history of undergoing in utero procedure while a fetus
- V15.21: Personal history of undergoing in utero procedure during pregnancy


In [43]:
reload(sdu)

<module 'structured_data_utils' from '/mnt/cbds_homes/ecarlson/Notebooks/mit_frequent_fliers/mit-team-code/software/notebooks/structured_data_utils.py'>

In [44]:
sdu.print_icd9_tree(diags[8]['clean_icd9_code'])

Parents:
- ROOT: ROOT
- V10-V19: PERSONS WITH POTENTIAL HEALTH HAZARDS RELATED TO PERSONAL AND FAMILY HISTORY 
- V15: Other personal history presenting hazards to health
- V15.2: Surgery to other organs

-> V15.2: Surgery to other organs

Children:
- V15.29: Surgery to other organs
- V15.22: Personal history of undergoing in utero procedure while a fetus
- V15.21: Personal history of undergoing in utero procedure during pregnancy


In [45]:
sdu.get_icd9_levels(diags[8]['clean_icd9_code'], 5)

['V10-V19', 'V15', 'V15.2']

## Gather the data

In [46]:
reload(sdu)

<module 'structured_data_utils' from '/mnt/cbds_homes/ecarlson/Notebooks/mit_frequent_fliers/mit-team-code/software/notebooks/structured_data_utils.py'>

In [47]:
found_notes = comb_dat.loc[comb_dat['row_id_m3'].notnull()].\
    groupby(['subject_id', 'md5', 'row_id_m3']).count()['total_m3_distance'].index.tolist()

In [48]:
note_info = {}
for idx in found_notes:
    note_meta = sdu.get_note_metadata(conn, idx[2])
    note_diag = sdu.get_hadm_diagnoses(conn, note_meta['hadm_id'])
    dat = {'meta': note_meta, 'diagnoses': note_diag}
    note_info[idx] = dat

In [49]:
note_info[[k for k in note_info.keys()][0]]

{'diagnoses': [{'clean_icd9_code': '410.71',
   'hadm_id': 172993,
   'icd9_code': '41071',
   'known_icd9_code': False,
   'long_title': 'Subendocardial infarction, initial episode of care',
   'seq_num': 1,
   'short_title': 'Subendo infarct, initial',
   'subject_id': 11590},
  {'clean_icd9_code': '398.91',
   'hadm_id': 172993,
   'icd9_code': '39891',
   'known_icd9_code': True,
   'long_title': 'Rheumatic heart failure (congestive)',
   'seq_num': 2,
   'short_title': 'Rheumatic heart failure',
   'subject_id': 11590},
  {'clean_icd9_code': '396.3',
   'hadm_id': 172993,
   'icd9_code': '3963',
   'known_icd9_code': True,
   'long_title': 'Mitral valve insufficiency and aortic valve insufficiency',
   'seq_num': 3,
   'short_title': 'Mitral/aortic val insuff',
   'subject_id': 11590},
  {'clean_icd9_code': '397.0',
   'hadm_id': 172993,
   'icd9_code': '3970',
   'known_icd9_code': True,
   'long_title': 'Diseases of tricuspid valve',
   'seq_num': 4,
   'short_title': 'Tricuspid

In [50]:
note_codes = []
note_meta = []
unknown_codes = set()
for k, note_dat in note_info.items():
    subject_id, md5, row_id = k

    meta = note_dat['meta'].copy()
    meta['subject_id'] = subject_id
    meta['md5'] = md5
    meta['note_row_id'] = row_id
    note_meta.append(meta)

    diagnoses = note_dat['diagnoses']
    if diagnoses is not None:
        for diag in diagnoses:
            new_code = {
                'subject_id': subject_id,
                'md5': md5,
                'note_row_id': row_id,
                'level': 'source',
                'code': diag['icd9_code']
            }
            note_codes.append(new_code)

            if diag['known_icd9_code']:
                levels = sdu.get_icd9_levels(diag['clean_icd9_code'])
                for ind, lev_code in enumerate(levels):
                    new_code = {
                        'subject_id': subject_id,
                        'md5': md5,
                        'note_row_id': row_id,
                        'level': ind,
                        'code': lev_code
                    }
                    note_codes.append(new_code)

            else:
                if diag['icd9_code'] not in unknown_codes:
                    unknown_codes.add(diag['icd9_code'])
                    logger.info('Unknown code ({}) for subject ({})'.format(diag['icd9_code'], subject_id))

2016-10-24 16:37:32,527 - root - INFO - Unknown code (41071) for subject (11590)
2016-10-24 16:37:32,575 - root - INFO - Unknown code (40391) for subject (11590)
2016-10-24 16:37:32,646 - root - INFO - Unknown code (41072) for subject (14342)
2016-10-24 16:37:32,647 - root - INFO - Unknown code (2859) for subject (14342)
2016-10-24 16:37:32,657 - root - INFO - Unknown code (2800) for subject (8217)
2016-10-24 16:37:32,662 - root - INFO - Unknown code (53200) for subject (8217)
2016-10-24 16:37:32,709 - root - INFO - Unknown code (2851) for subject (23829)
2016-10-24 16:37:32,733 - root - INFO - Unknown code (E8498) for subject (23829)
2016-10-24 16:37:32,739 - root - INFO - Unknown code (25040) for subject (23829)
2016-10-24 16:37:32,744 - root - INFO - Unknown code (28521) for subject (23829)
2016-10-24 16:37:32,745 - root - INFO - Unknown code (25070) for subject (23829)
2016-10-24 16:37:32,755 - root - INFO - Unknown code (E8497) for subject (23829)
2016-10-24 16:37:32,779 - root - 

In [51]:
len(unknown_codes)

375

In [52]:
note_codes_df = pd.DataFrame.from_records(note_codes)
note_codes_df.head(5)

,code,level,md5,note_row_id,subject_id
0,41071,source,be74552c73a0f9895c4f372763054d26,1414073.0,11590
1,39891,source,be74552c73a0f9895c4f372763054d26,1414073.0,11590
2,390-459,0,be74552c73a0f9895c4f372763054d26,1414073.0,11590
3,393-398,1,be74552c73a0f9895c4f372763054d26,1414073.0,11590
4,398,2,be74552c73a0f9895c4f372763054d26,1414073.0,11590


In [57]:
output_path = pl.Path(path_config['repo_data_dir']).joinpath('notes_icd9_codes_{}.csv'.format(time_str))
logger.info(output_path)
note_codes_df.to_csv(output_path.as_posix(), index=False)

2016-10-24 16:41:53,258 - root - INFO - ../../data/notes_icd9_codes_2016-10-24-16-35.csv


In [54]:
note_meta_df = pd.DataFrame.from_records(note_meta)
note_meta_df.head(5)

,cgid,chartdate,charttime,hadm_id,md5,note_row_id,storetime,subject_id
0,17770.0,2154-06-03,2154-06-03 17:30:00,172993.0,be74552c73a0f9895c4f372763054d26,1414073.0,2154-06-03 17:51:00,11590
1,17698.0,2183-07-28,2183-07-28 05:41:00,116105.0,2bd0c96855c6107be79d0150e1f121e7,1449706.0,2183-07-28 05:53:00,14342
2,NaN,2170-02-13,NaT,122710.0,bd4bf8040238e3e2cdd7466692defe73,47105.0,NaT,8217
3,18469.0,2175-06-07,2175-06-07 05:39:00,196691.0,6d20d9b6d3cfdc3fc9e8a72fbab0f697,1573953.0,2175-06-07 06:27:00,23829
4,17079.0,2125-04-27,2125-04-27 20:51:00,133059.0,d35003faa86241e60396014264b14a4d,1264491.0,2125-04-27 21:03:00,305


In [58]:
output_path = pl.Path(path_config['repo_data_dir']).joinpath('mimic3_note_metadata_{}.csv'.format(time_str))
logger.info(output_path)
note_meta_df.to_csv(output_path.as_posix(), index=False)

2016-10-24 16:41:54,930 - root - INFO - ../../data/mimic3_note_metadata_2016-10-24-16-35.csv
